In [1]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import time
import sys
import os

import seaborn as sns
import gensim

import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append('/home/leey/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/Documents/Data/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search, run_tm
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import *
from utils.tm_mgmt import update_topic_scores
from utils.text import *
from django.db.models import Q, Count, Func, F, Sum, Value, Case, When, IntegerField

In [ ]:
def clean_text(text):

    text = text.replace('\r', '\n')
    text = text.replace(u'\xa0', ' ')
    text = text.replace(u'\x96', '-')
    text = text.replace(u'\xad', '-')
    text = text.replace(u'\u2014', '–')
    # text = text.replace(u'\u2013', '–')
    return text


### Finding agenda items

In [2]:
# finding the agenda items of the utterances that appear in the search (id = 89)
utterances = pm.Utterance.objects.filter(search_matches__pk=89)
#agenda_items = pm.AgendaItem.objects.filter()

In [7]:
agenda_items = []
for ut in utterances:
    try:
        agenda_items.append(ut.agenda_item.title)
    except AttributeError:
        pass

In [42]:
import pickle
fileObject = open('agenda_items_list.pkl','wb')
pickle.dump(agenda_items,fileObject)   
fileObject.close()

In [13]:
df_agenda_items = pd.DataFrame(agenda_items)
df_agenda_items.columns = ['Agenda Item']

In [14]:
df_agenda_items

,Agenda Item
0,(Fortsetzung der Aussprache)
1,(Fortsetzung der Aussprache)
2,(Fortsetzung der Aussprache)
3,(Fortsetzung der Aussprache)
4,(Fortsetzung der Aussprache)
5,(Fortsetzung der Aussprache)
6,(Fortsetzung der Aussprache)
7,(Fortsetzung der Aussprache)
8,"Beratung des Antrags der Fraktionen CDU/CSU, S..."
9,"Beratung des Antrags der Fraktionen CDU/CSU, S..."


In [25]:
# extract unique agenda items only 
agenda_items_unique = list(set(agenda_items))
df_agenda_items_unique = pd.DataFrame(agenda_items_unique)
df_agenda_items_unique.columns = ['Agenda Item']

In [21]:
df_agenda_items_unique

,Agenda Item
0,l) Beratung der Beschlussempfehlung und des Be...
1,22. Beratung des Antrags der Abgeordneten Nico...
2,b) Beratung der Unterrichtung durch den Präsid...
3,Beratung des Antrags der Abgeordneten Dr. Alex...
4,hier: Einzelplan 09 Bundesministerium für Wir...
5,Beratung des Antrags der Abgeordneten Sabine Z...
6,hier: Einzelplan 23 Bundesministerium für wir...
7,Beratung des Antrags der Abgeordneten ­Dr. Bru...
8,Beratung des Antrags der Abgeordneten Frank Mü...
9,d) Beratung der Beschlussempfehlung und des Be...


In [34]:
from nltk.tokenize import sent_tokenize, word_tokenize

nltk_words = word_tokenize(agenda_items_unique[0]) 
display(f"Tokenized words: {nltk_words}")

"Tokenized words: ['l', ')', 'Beratung', 'der', 'Beschlussempfehlung', 'und', 'des', 'Berichts', 'des', 'Ausschusses', 'für', 'Umwelt', ',', 'Naturschutz', 'und', 'nukleare', 'Sicherheit', '(', '16', '.', 'Ausschuss', ')', 'zu', 'dem', 'Antrag', 'der', 'Abgeordneten', 'Lisa', 'Badum', ',', 'Dr.', 'Julia', 'Verlinden', ',', 'Dr.', 'Ingrid', 'Nestle', ',', 'weiterer', 'Abgeordneter', 'und', 'der', 'Fraktion', 'BÜNDNIS', '90/DIE', 'GRÜNEN']"

In [37]:
agenda_items_unique_lower = [word.lower() for word in agenda_items_unique]

In [38]:
tokens_list = []
for words in agenda_items_unique_lower:
    tokens = word_tokenize(words)
    tokens_list.append(tokens)

In [39]:
tokens_list

[['l',
  ')',
  'beratung',
  'der',
  'beschlussempfehlung',
  'und',
  'des',
  'berichts',
  'des',
  'ausschusses',
  'für',
  'umwelt',
  ',',
  'naturschutz',
  'und',
  'nukleare',
  'sicherheit',
  '(',
  '16.',
  'ausschuss',
  ')',
  'zu',
  'dem',
  'antrag',
  'der',
  'abgeordneten',
  'lisa',
  'badum',
  ',',
  'dr.',
  'julia',
  'verlinden',
  ',',
  'dr.',
  'ingrid',
  'nestle',
  ',',
  'weiterer',
  'abgeordneter',
  'und',
  'der',
  'fraktion',
  'bündnis',
  '90/die',
  'grünen'],
 ['22.',
  'beratung',
  'des',
  'antrags',
  'der',
  'abgeordneten',
  'nicola',
  'beer',
  ',',
  'katja',
  'suding',
  ',',
  'dr.',
  'jens',
  'brandenburg',
  '(',
  'rhein-neckar',
  ')',
  ',',
  'weiterer',
  'abgeordneter',
  'und',
  'der',
  'fraktion',
  'der',
  'fdp'],
 ['b',
  ')',
  'beratung',
  'der',
  'unterrichtung',
  'durch',
  'den',
  'präsidenten',
  'des',
  'deutschen',
  'bundestages'],
 ['beratung',
  'des',
  'antrags',
  'der',
  'abgeordneten',
  '